In [1]:
import sqlite3
import random
import requests
import numpy as np
import pandas as pd
import sqlite3
import pandas as pd
import numpy as np
import time
import datetime
from datetime import timedelta
from datetime import datetime as dt
import multiprocessing as mp
#from Data_Management.data_update_version_for_coinlist import data_update_version_for_coinlist
from Data_Management.data_update import data_update

from binance_f import RequestClient
from binance_f.constant.test import *
from binance_f.base.printobject import *
from binance_f.model.constant import *
g_api_key='0acce0f73ddd167d802df84cc7073b28f9b7ef24702557d134b546faf9e18e39'
g_secret_key='f2f93c0be4d5ec09d6ca28ef0dd96c200c434a0684762b500b607a21375f2c30'
request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key, url="https://testnet.binancefuture.com")

In [2]:
#import models
from Model.resistance_4hrs_function import resistance_4hrs
from Model.null_model import null_model_v2
model_list = [null_model_v2, resistance_4hrs]
model_name_list = list(map(lambda x: str(x).split()[1],model_list))

# Data update

In [3]:
with open('coinlist.txt') as f:
    coinlist = f.read().splitlines()

In [66]:
#10-18mins
pool_for_data_processing = {}
for i in range(len(coinlist)): #155
    pool_for_data_processing[i] = mp.Process(target=data_update, args=(coinlist[i],))
    
for i in range(len(coinlist)):
    pool_for_data_processing[i].start()
for i in range(len(coinlist)):
    pool_for_data_processing[i].join()

# Implement model


4. 一分钟级别交易 binance实时数据
5. 把半个小时拿掉，6.30判断7.00下单

In [7]:
conn = sqlite3.connect('./testing/binancef_database_updated')
c = conn.cursor()


def get_resampled_df_from_db(symbol,interval): #interval in str type: '1H','2H','30MIN',etc.
    sql = 'SELECT * FROM '+ symbol
    c = conn.cursor()
    c.execute(sql)    
    data = c.fetchall()
    column_des = c.description
    column_names = [column_des[i][0] for i in range(len(column_des))]
    df = pd.DataFrame([list(i) for i in data],columns=column_names)

    df = df.drop_duplicates(['timestamp']).reset_index(drop = True)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')
    
    return df.resample(interval).mean()

In [8]:
#139个币可下载
def get_df_for_all_coins(resample_interval, determinant_interval): #'30MIN', 48

    df_for_all_coins = pd.DataFrame(columns=['amount','mark_price','funding_rate','open_interest','liquidation_value_usd','volume_usd','symbol'])
    No_failure = 0
    for coin in coinlist:
        try:
            df = get_resampled_df_from_db(coin,resample_interval)[-1*determinant_interval:] #single asset
            df['symbol'] = coin
            df_for_all_coins = pd.concat([df_for_all_coins, df]) #
        except:
            No_failure += 1
            print(f'failed to retrive {coin}')
            
    df_for_all_coins['liquidation_value_usd'] = df_for_all_coins['liquidation_value_usd'].fillna(0)
    df_for_all_coins = df_for_all_coins.dropna() #删掉全是空值的

    print(No_failure)
    return df_for_all_coins

In [9]:
df_for_all_coins = get_df_for_all_coins('1H',36)

failed to retrive 1inchusdt
failed to retrive 1000shibusdt
failed to retrive 1000xecusdt
failed to retrive scusdt
failed to retrive icpusdt
failed to retrive tlmusdt
failed to retrive akrousdt
failed to retrive dodousdt
failed to retrive ancusdt
failed to retrive lunausdt
failed to retrive yfiiusdt
failed to retrive 1000bttcusdt
failed to retrive keepusdt
failed to retrive nuusdt
failed to retrive bttusdt
failed to retrive bzrxusdt
failed to retrive btcstusdt
failed to retrive dotecousdt
failed to retrive lendusdt
19


In [10]:
df_for_all_coins

,amount,mark_price,funding_rate,open_interest,liquidation_value_usd,volume_usd,symbol
2022-08-07 20:00:00,606.368667,23075.516059,-0.000010,1.109942e+05,64170.074816,1.399609e+07,btcusdt
2022-08-07 21:00:00,461.704500,23134.808102,-0.000014,1.110861e+05,7918.428467,1.068206e+07,btcusdt
2022-08-07 22:00:00,403.315867,23112.272326,-0.000014,1.114874e+05,4477.495286,9.319613e+06,btcusdt
2022-08-07 23:00:00,374.378867,23061.699485,0.000010,1.114720e+05,53122.386930,8.629448e+06,btcusdt
2022-08-08 00:00:00,457.354233,23073.868402,0.000026,1.112252e+05,28588.303743,1.054624e+07,btcusdt
...,...,...,...,...,...,...,...
2022-08-09 03:00:00,177239.180000,1.905366,-0.000052,2.431120e+07,1746.969521,3.378385e+05,opusdt
2022-08-09 04:00:00,186263.970000,1.908902,-0.000056,2.430323e+07,1218.678541,3.561742e+05,opusdt
2022-08-09 05:00:00,203187.623333,1.897282,-0.000002,2.407250e+07,1943.928108,3.855417e+05,opusdt
2022-08-09 06:00:00,271968.870000,1.888129,0.000019,2.409840e+07,22184.378126,5.126712e+05,opusdt


In [9]:
def get_order_book_sql(df_for_all_coins, model): #generate signal for all coins using a single model
    #apply model 
    signal_df = df_for_all_coins.groupby('symbol', sort = False).apply(model)   
    signal_df = signal_df.drop(columns=['amount','mark_price','funding_rate','open_interest','liquidation_value_usd','volume_usd']) #['symbol','open_signal','close_signal',other signals] 
    types = list(signal_df)[1:] #remove 'symbol', ['open_signal','close_signal',other signals] 
    historical_order_book = pd.DataFrame(columns=['symbol','signal_no','signal_type']) #create empty df
    for type in types:
        type_df = signal_df.dropna(subset=[type]).rename(columns={type:'signal_no'}) #signal_df[signal_df[type]!= 0].rename(columns={type:'signal_no'})
        type_df['signal_type'] = type
        historical_order_book = pd.concat([historical_order_book, type_df], join='inner') # drop irrelevant columns

    historical_order_book = historical_order_book.sort_index().reset_index().rename(columns={'index':'timestamp'})
    model_name = str(model).split()[1]
    
    c.execute(f'CREATE TABLE IF NOT EXISTS {model_name} (timestamp, symbol, signal_no, signal_type)')
    conn.commit()
    historical_order_book.to_sql(model_name, conn, if_exists='append', index = False)
    c.execute(f'''  
    SELECT * FROM {model_name}
            ''')
    print(f'finish into_sql for {model_name}') 

In [11]:
conn = sqlite3.connect('./testing/historical_order_books')
c = conn.cursor()

In [11]:
for model in model_list:
    get_order_book_sql(df_for_all_coins,model)

finish into_sql for null_model_v2
finish into_sql for resistance_4hrs


————————————————————————————————————————————————————————————————————————————————————————————————————
testing

In [12]:
def get_df_from_db(model): #interval in str type: '1H','2H','30MIN',etc.
    model_name = str(model).split()[1]
    sql = 'SELECT * FROM '+ model_name
    c = conn.cursor()
    c.execute(sql)    
    data = c.fetchall()
    column_des = c.description
    column_names = [column_des[i][0] for i in range(len(column_des))]
    df = pd.DataFrame([list(i) for i in data],columns=column_names)
    
    return df

In [13]:
get_df_from_db(resistance_4hrs)

,timestamp,symbol,signal_no,signal_type
0,2022-08-07 00:00:00,rsrusdt,-1,open_signal
1,2022-08-07 00:00:00,zilusdt,-1,open_signal
2,2022-08-07 04:00:00,zilusdt,2022-08-07 00:00:00,close_signal
3,2022-08-07 04:00:00,rsrusdt,2022-08-07 00:00:00,close_signal
4,2022-08-07 06:00:00,solusdt,-1,open_signal
...,...,...,...,...
119,2022-08-09 02:00:00,sushiusdt,2022-08-08 22:00:00,close_signal
120,2022-08-09 02:00:00,defiusdt,2022-08-08 22:00:00,close_signal
121,2022-08-09 03:00:00,dashusdt,2022-08-08 23:00:00,close_signal
122,2022-08-09 06:00:00,btcdomusdt,2022-08-09 02:00:00,close_signal


In [14]:
get_df_from_db(null_model_v2)

,timestamp,symbol,signal_no,signal_type
0,2022-08-06 20:00:00,uniusdt,-1,open_signal
1,2022-08-06 20:00:00,rsrusdt,-1,open_signal
2,2022-08-06 20:00:00,unfiusdt,1,open_signal
3,2022-08-06 20:00:00,chrusdt,1,open_signal
4,2022-08-06 20:00:00,flowusdt,-1,open_signal
...,...,...,...,...
6921,2022-08-09 07:00:00,storjusdt,2022-08-09 03:00:00,close_signal
6922,2022-08-09 07:00:00,apeusdt,2022-08-09 03:00:00,close_signal
6923,2022-08-09 07:00:00,icxusdt,2022-08-09 03:00:00,close_signal
6924,2022-08-09 07:00:00,ensusdt,2022-08-09 03:00:00,close_signal


——————————————————————————————————————————————————————————————————————————————————————
废料

In [19]:
def get_signal(df_for_all_coin, model): #generate signal for all coins using a single model
    for df in df_for_all_coin: #[btc_df,eth_df,...]
        try:
            symbol = df['symbol'][0] #get coin name：'btcusdt' str
            signal_df = model(df) #apply model 
            signal = signal_df.iloc[-1:]['open_signal'] #7:30-8:00的总信号 #8:00 
            total_signal_df[str(model).split()[1]][symbol] = signal
        except:
            print(f'fail to get {coin}') #9:00-15:00, 15:00-21:00, 24*6hr, 12个点
        
for model in model_list:
    get_signal(df_for_all_coin, model) #7-8mins
    
total_signal_df['signal_sum'] = sum(total_signal_df[i] for i in model_name_list)
total_signal_df.head(50)

c:\Users\hzixu\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\hzixu\Desktop\algo trading\Algo_strats\Model\resistance_4hrs_function.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[f'K{interval}'][i] = ((df1['mark_price'][i+interval] - df1['mark_price'][i])/df1['mark_price'][i])*1000 #i is the start of ups and downs


,null_model_v1,resistance_4hrs,signal_sum
btcusdt,2,0,2
ethusdt,2,0,2
bchusdt,-1,0,-1
xrpusdt,2,0,2
eosusdt,2,0,2
ltcusdt,7,0,7
trxusdt,-1,0,-1
etcusdt,3,0,3
linkusdt,2,0,2
xlmusdt,-6,0,-6


In [58]:
'''multiprocessing 反而更慢了
pool_for_generating_signal = {}

for model in model_list:
    pool_for_generating_signal[model] = mp.Process(target=get_signal, args=(df_for_all_coin, model)) #process for every model applied to every coin

for model in model_list:
    pool_for_generating_signal[model].start()
for model in model_list:
    pool_for_generating_signal[model].join()


    
pool_for_generating_signals = {}
for model in model_list:
    pool_for_generating_signals[model] = {}
    for df in df_for_all_coin:
        try:
            pool_for_generating_signals[model][df['symbol'][0]] = mp.Process(target=get_signal, args=(df,model)) #process for every model applied to every coin
        except:
            pass

for sub_pool in pool_for_generating_signals:
    for process in pool_for_generating_signals[sub_pool]:
        pool_for_generating_signals[sub_pool][process].start()

for sub_pool in pool_for_generating_signals:
    for process in pool_for_generating_signals[sub_pool]:
        pool_for_generating_signals[sub_pool][process].join()
'''

1. signal 和仓位
2. take_profit, stop_loss
3. 定时早八

In [ ]:
new_df = pd.DataFrame(columns=list(original_df.columns)+['high_price','low_price','open_price','close_price'], index=list(range(48))).fillna(0.0).drop(columns=['funding_rate','side']) #0.0 for float type :(
for i in range(48):
    new_df['timestamp'][i] = original_df['timestamp'][i*30]
    new_df['amount'][i] = original_df.iloc[i*30:(i+1)*30]['amount'].sum() #和
    new_df['mark_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].mean() #均价
    new_df['high_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].max() #高价
    new_df['low_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].min() #低价
    new_df['open_price'][i] = original_df['mark_price'][i*30] #开盘价
    new_df['close_price'][i] = original_df['mark_price'][(i+1)*30-1] #收盘价
    new_df['volume_usd'][i] = original_df.iloc[i*30:(i+1)*30]['volume_usd'].sum() #和
    new_df['open_interest'][i] = original_df['open_interest'][(i+1)*30-1] #最后一项？
    new_df['liquidation_value_usd'][i] = original_df.iloc[i*30:(i+1)*30]['liquidation_value_usd'].sum()
new_df

In [ ]:
#手动resample
def get_manipulated_df(symbol):
    original_df = get_df_from_db(symbol).drop_duplicates(['timestamp']).reset_index(drop=True)
    print(f'successfully get original df for {coin}')
    new_df = pd.DataFrame(columns=list(original_df.columns)+['high_price','low_price','open_price','close_price'], index=list(range(48))).fillna(0.0).drop(columns=['funding_rate','side']) #0.0 for float type :(
    for i in range(48):
        new_df['timestamp'][i] = original_df['timestamp'][i*30]
        new_df['amount'][i] = original_df.iloc[i*30:(i+1)*30]['amount'].sum() #和
        new_df['mark_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].mean() #均价
        new_df['high_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].max() #高价
        new_df['low_price'][i] = original_df.iloc[i*30:(i+1)*30]['mark_price'].min() #低价
        new_df['open_price'][i] = original_df['mark_price'][i*30] #开盘价
        new_df['close_price'][i] = original_df['mark_price'][(i+1)*30-5] #收盘价
        new_df['volume_usd'][i] = original_df.iloc[i*30:(i+1)*30]['volume_usd'].sum() #和
        new_df['open_interest'][i] = original_df['open_interest'][(i+1)*30-5] #最后一项？
        new_df['liquidation_value_usd'][i] = original_df.iloc[i*30:(i+1)*30]['liquidation_value_usd'].sum()
    print('successfully get new df')
    return new_df

In [ ]:
#跑得快但是跑不通 烦死了
'''
pool = mp.Pool()
res = pool.map(data_update, coinlist)
'''

In [ ]:
#手动设pool*2
'''
pool = {}
core_set = 8
for i in range(core_set-1):
    pool[i] = mp.Process(target=data_update, args=(coinlist[155//core_set*i:155//core_set*(i+1)],))
pool[core_set-1] = mp.Process(target=data_update, args=(coinlist[155//core_set*(i+1):],)) #最后一段长度不一样
for i in range(core_set):
    pool[i].start()
for i in range(core_set):
    pool[i].join()
'''